# Simple Tensorflow Models

In this Notebook we will be working on some simple tensorflow models for the classification. We will try out:
- Baseline (a threshhold model to compare against)
- Feed-forward network
- LSTM network
- Auto-encoder for anomaly detection

In [1]:
import os

import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt


import tensorflow as tf

from sklearn.model_selection import train_test_split

# Colors using scheme
c = ["#" + i for i  in "264653-2a9d8f-e9c46a-f4a261-e76f51".split("-")]

2022-01-13 06:41:29.035745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 06:41:29.035791: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_trace_high = pd.read_csv("/mnt/c/Users/johan/Desktop/CMT_project/data/traces_high.txt", sep = " ", header = None)
data_trace_low  = pd.read_csv("/mnt/c/Users/johan/Desktop/CMT_project/data/traces_low.txt",  sep = " ", header = None)

In [3]:
rand_seed  = 1969
test_split = 0.2


high_labels = np.ones(data_trace_high.shape[0])
low_labels  = np.zeros(data_trace_low.shape[0])

data        = pd.concat([data_trace_high, data_trace_low])
labels      = np.concatenate([high_labels,     low_labels])


X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = test_split)